In [5]:
import pandas as pd
import numpy as np

import requests

from datetime import datetime, timedelta
from pprint import pprint



### EJEMPLO


In [18]:
lang    =  "es"
category = "balance"
widget   = "balance-electrico"
query   = "start_date=2018-01-01T00:00&end_date=2018-01-07T23:59&time_trunc=day"


In [19]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [20]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()
pprint(data)

{'data': {'attributes': {'description': 'Balance eléctrico: asignación de '
                                        'unidades de producción según '
                                        'combustible principal. La producción '
                                        'neta de las instalaciones no '
                                        'renovables e hidráulicas UGH tiene '
                                        'descontados los consumos propios de '
                                        'las instalaciones. En dichos tipos de '
                                        'producción una generación negativa '
                                        'indica que la electricidad horaria '
                                        'consumida para los usos de las '
                                        'plantas excede la producción horaria '
                                        'de las instalaciones.',
                         'last-update': '2019-06-12T17:40:40.000+02:00',
              

# OBTENCIÓN DE DATOS DE BALANCE




In [6]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [13]:
lang    =  input('¿En español (es) o en inglés (en)?') 
category = input('¿Balance, demanda, generación o intercambio?') 
widget   = input('¿Qué widget quieres?')


restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

#balance_datos(lang, category, widget, restaDia, input_año)

In [14]:
#def balance_datos(lang, category, widget, restaDia, input_año):

now = datetime.now()
ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"



endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()



lista_nombres = list()
lista_tipos = list()
lista_valores = list()
lista_porcentajes = list()
lista_fechas = list()
lista_dias = list()
lista_meses = list()
lista_años = list()


for dato in data['included']: 
    for info in dato['attributes']['content']:
        nombre = info['type']

        tipo = info['groupId']


        for i in info['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)
            dia = fecha.strftime('%d')
            mes = fecha.strftime('%m')
            año = fecha.strftime('%Y')

            fecha =  fecha.strftime("%d/%m/%Y")

        
            
            
            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)
            lista_dias.append(dia)
            lista_meses.append(mes)
            lista_años.append(año)

df_balance = pd.DataFrame()
df_balance['nombre']               = lista_nombres
df_balance['tipo de energía']      = lista_tipos
df_balance['Valores']              = lista_valores
df_balance["Porcentaje"]           = lista_porcentajes
df_balance["Fecha actualización"]  = lista_fechas
df_balance["dia"]                  = lista_dias               
df_balance["mes"]                  = lista_meses               
df_balance["año"]                  = lista_años               




    #return df_balance  


In [32]:
#balance_datos(lang, category, widget, restaDia, input_año).to_csv('balance_electrico.csv')

In [15]:
df_balance.head(5).to_dict()

{'nombre': {0: 'Hidráulica',
  1: 'Hidráulica',
  2: 'Hidráulica',
  3: 'Hidráulica',
  4: 'Hidráulica'},
 'tipo de energía': {0: 'Renovable',
  1: 'Renovable',
  2: 'Renovable',
  3: 'Renovable',
  4: 'Renovable'},
 'Valores': {0: 78649.0, 1: 73717.8, 2: 51906.1, 3: 38797.6, 4: 43983.0},
 'Porcentaje': {0: 0.2327651272457748,
  1: 0.23665761392175402,
  2: 0.122386259902673,
  3: 0.07617413943568867,
  4: 0.09587972767436792},
 'Fecha actualización': {0: '23/09/2024',
  1: '24/09/2024',
  2: '25/09/2024',
  3: '26/09/2024',
  4: '27/09/2024'},
 'dia': {0: '23', 1: '24', 2: '25', 3: '26', 4: '27'},
 'mes': {0: '09', 1: '09', 2: '09', 3: '09', 4: '09'},
 'año': {0: '2024', 1: '2024', 2: '2024', 3: '2024', 4: '2024'}}

In [16]:
df_balance.head(5)

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización,dia,mes,año
0,Hidráulica,Renovable,78649.0,0.232765,23/09/2024,23,09,2024
1,Hidráulica,Renovable,73717.8,0.236658,24/09/2024,24,09,2024
2,Hidráulica,Renovable,51906.1,0.122386,25/09/2024,25,09,2024
3,Hidráulica,Renovable,38797.6,0.076174,26/09/2024,26,09,2024
4,Hidráulica,Renovable,43983.0,0.095880,27/09/2024,27,09,2024


In [17]:
df_balance.to_csv('balance_electrico.csv')